Design a database that have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [13]:
import pandas as pd
import numpy as np

# Read in bank_marketing.csv as a pandas DataFrame.
marketing_data = pd.read_csv('bank_marketing.csv')

# Split the data into three DataFrames
client = marketing_data[["client_id", "age", "job", "marital", "education", 
             "credit_default", "housing", "loan"]]
campaign = marketing_data[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]
economics = marketing_data[["client_id", "emp_var_rate", "cons_price_idx", 
                "euribor3m", "nr_employed"]]

# Rename columns 
client = client.rename(columns = {"client_id" : "id"})

economics = economics.rename(columns = {"euribor3m" : "euribor_three_months", 
                                        "nr_employed" : "number_employed"})

campaign = campaign.rename(columns = {"duration" : "contact_duration", 
                                        "previous" : "previous_campaign_contacts", 
                                        "y" : "campaign_outcome", 
                                        "poutcome" : "previous_outcome", 
                                        "campaign" : "number_contacts"})

# Clean the education column in client_data DataFrame
client['education'] = client['education'].str.replace('.', '_')

# Replace null values in education with numpy.nan
client['education'] = client['education'].replace("unknown", np.NaN)

# Remove periods from the "job" column
client['job'] = client['job'].str.replace('.', '')

# Convert values to binary in previous_outcome and campaign_outcome columns
campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", np.NaN)

campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, "failure": 0})

campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, "no": 0})

# Add a column called campaign_id in campaign, where all rows have a value of 1
campaign['campaign_id'] = 1

# Create a datetime column called last_contact_date
campaign['last_contact_date'] = '2022-' + campaign['month'].astype(str) + '-' + campaign['day'].astype(str)

campaign['last_contact_date'] = pd.to_datetime(campaign["last_contact_date"], format="%Y-%b-%d")

# Drop unneccessary columns
campaign.drop(columns=["month", "day"])

# Save the three DataFrames to csv files
client.to_csv('client.csv', index = False)
campaign.to_csv('campaign.csv', index = False)
economics.to_csv('economics.csv', index = False)

# Store database_design
client_table = """CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER"""

campaign_table = """CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER"""

economics_table = """CREATE TABLE economics (
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER"""